Transformers are deep learning architectures introduced by Google in 2017 that are designed to process sequential data for downstream tasks such as translation, question answering or text summarization.

Let's first talk about text generation.
The text generation capabilities of GPT-2 is one of the most popular Transformer architectures usable by a broader audience

In [1]:
import tensorflow as tf

One of the advantages of the transformer library and a reason for its popularity is how easily we can download a specific model.

In [2]:
! pip install transformers

In [3]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


It is usually a good idea to fix the random seed to make sure the results are reproducable. As can be seen from the above result, we did download all the pretrained model from the transformers libray. It will help us to produce the code faster with more accuracy.

In [5]:
# settings

#for reproducability
SEED = 34
tf.random.set_seed(SEED)

#maximum number of words in output text
MAX_LEN = 70

The next step is we need to decode and it is one of the most important decisions when using the GPT-2 model.

We will use greedy search, the word with the highest probability is predicted as the next word in the sequence.

In [6]:
input_sequence = "There are times when we are really tired of people but we feel lonely too"

In [7]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(input_sequence, return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = GPT2.generate(input_ids, max_length = MAX_LEN)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
There are times when we are really tired of people but we feel lonely too. We are not alone. We are not alone. We are not alone. We are not alone. We are not alone. We are not alone. We are not alone. We are not alone. We are not alone. We are not alone. We are not alone


As can be seen from the above result, the model starts repeating itself, because the high-probability words mask the less-likely ones so they can not explore more diverse combinations.

A simple solution for that is using beam search, we can keep track of the alternative variants so that more comparisons are possible.

In [8]:
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids,
    max_length = MAX_LEN,
    num_beams = 5,
    no_repeat_ngram_size = 2,
    num_return_sequences = 5,
    early_stopping = True
)

print('')
print("Output:\n" + 100 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
----------------------------------------------------------------------------------------------------
0: There are times when we are really tired of people but we feel lonely too.

"It's not that we don't want to talk to people, it's just that sometimes we just feel like we can't."
1: There are times when we are really tired of people but we feel lonely too.

"It's not that we don't want to talk to them, it's just that they're not there for us. We want them to be there with us."
2: There are times when we are really tired of people but we feel lonely too.

"It's not that we don't want to talk to them, it's just that they're not there."
3: There are times when we are really tired of people but we feel lonely too.

"It's not that we don't want to talk to them, it's just that they're not there for us. We want them to be there with us, but they aren't."
4: There are times when we are really tired of people but we feel lonely too.

"It's not that we don't want to talk to them, it's 

As can be seen from the above result, the result is now more diverse, the message is still the same, but at least the formulations look a little different from a style point of view.

The next step is we can explore sampling- indeterministic decoding. Instead of following a strict path to find the end text with the highest probabbility, we can rather randomly pick the next word by its conditional probability distribution. This approach risks producing incoherent samplings so we can make sue of the temparature parameter which affects the probability mass distribution.